In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-import operator

# M. Kempka, T.Sternal, M.Wydmuch, Z.Boztoprak
# Januar 2021
import sys
import os

#import horovod
from mpi4py import MPI
import numpy as np
import vizdoom as vzd
import itertools as it

In [2]:
import tensorflow as tf

In [3]:
import horovod.tensorflow as hvd
import skimage.color, skimage.transform
from tensorflow import keras
from tqdm import trange
from random import sample
from time import time, sleep
from collections import deque
import tensorflow_datasets as tfds
#import resnet
from tensorflow.keras import Model, Sequential, Input, losses, metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, ReLU

#tf.compat.v1.enable_eager_execution() #tf.compat.v1.disable_eager_execution() 
#tf.executing_eagerly()
import cv2

In [4]:
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [5]:

print("OS ENV", os.environ)

OS ENV environ({'SHELL': '/bin/bash', 'SESSION_MANAGER': 'local/IG-88:@/tmp/.ICE-unix/3307,unix/IG-88:/tmp/.ICE-unix/3307', 'QT_ACCESSIBILITY': '1', 'COLORTERM': 'truecolor', 'XDG_CONFIG_DIRS': '/etc/xdg/xdg-ubuntu:/etc/xdg', 'XDG_MENU_PREFIX': 'gnome-', 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated', 'CONDA_EXE': '/home/spillingvoid/anaconda3/bin/conda', '_CE_M': '', 'GNOME_SHELL_SESSION_MODE': 'ubuntu', 'SSH_AUTH_SOCK': '/run/user/1000/keyring/ssh', 'I_MPI_ROOT': '/home/spillingvoid/anaconda3', 'XMODIFIERS': '@im=ibus', 'DESKTOP_SESSION': 'ubuntu', 'FI_PROVIDER_PATH': '/home/spillingvoid/anaconda3/lib/libfabric/prov:/usr/lib64/libfabric', 'SSH_AGENT_PID': '3273', 'GTK_MODULES': 'gail:atk-bridge', 'PWD': '/home/spillingvoid', 'LOGNAME': 'spillingvoid', 'XDG_SESSION_DESKTOP': 'ubuntu', 'XDG_SESSION_TYPE': 'x11', 'CONDA_PREFIX': '/home/spillingvoid/anaconda3', 'MANPATH': '/home/spillingvoid/anaconda3/share/man:/home/spillingvoid/anaconda3/man:/home/spillingvoid/anaconda3/share/man:/ho

In [6]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True) 

In [7]:
tf.test.is_built_with_cuda()

True

In [8]:
os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_GPU_THREAD_COUNT'] = '2'

In [9]:
xla_compile=True

In [10]:
from tensorflow.python.client.device_lib import list_local_devices
devices = list_local_devices()
print(devices)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1437606969350997364
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9068007922568998344
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5793841152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4225983483377547173
physical_device_desc: "device: 0, name: GeForce GTX TITAN, pci bus id: 0000:04:00.0, compute capability: 3.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 3862560768
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15333409269675773363
physical_device_desc: "device: 1, name: Quadro M2000, pci bus id: 0000:03:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12432830187968532014
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"


In [11]:
#initilize Horovod
hvd.init()

In [12]:
tf.config.get_visible_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [13]:
#assign GPUs
gpus = tf.config.list_physical_devices('GPU')

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [15]:
print("GPU", tf.config.list_physical_devices('GPU'))

GPU [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [16]:
print("gpu_devices", gpus)

gpu_devices [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [17]:
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [18]:
#print("gpus", 'GPU')

In [19]:
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [20]:
print(gpu)

PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [21]:
#if gpus:
tf.config.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
    

In [22]:
print(tf.config.set_visible_devices(gpus[hvd.local_rank()], 'GPU'))

None


In [23]:
print("hvd rank", hvd.local_rank())

hvd rank 0


In [24]:
print("hvd size", hvd.size())

hvd size 1


In [25]:
#time GPU=0[ epoch 16 steps 512 batch 16 time{433 min}] 
#GPU=1 [epoch 16, 512 steps, 16 batch, 16time{<110 min}]
#GPU=1 [epoch 32, 2048 steps, 16 batch, 16 time{}]
#GPU=2 [epoch 16, steps 512, 16 batch, time{112.80 }]
#GPU=2 [epoch 16, steps 512, 32 batch, time{152 min}]
#GPU=2 w/horovod [epoch 16, steos 512, update 64, batch 16 time{37.94 minutes}] only 1 gpu seemed to be working
#GPU=2 w/horovod [epoch 16, steos 512, update 64, batch 16 time{}]

In [26]:
# Q-learning settings
learning_rate = 0.00000025
discount_factor = 0.99
replay_memory_size = 1000000 
num_train_epochs = 256 #256 
learning_steps_per_epoch = (2048 // hvd.size()) #2048
target_net_update_steps =  64 #1024

# NN learning settings
#BUFFER = len(extractDigits(*argv))
batch_size = 16
BATCH_SIZE_PER_REPLICA = 8
#GLOBALBATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync
EPOCHS = 8
epoch = 8
#x = [0]
K = keras.backend
# Training regime 
test_episodes_per_epoch = 4

# Other parameters
frames_per_action = 3
resolution = (45, 30)
shape = (1, 30, 45, 2)
episodes_to_watch = 2
num_action = 3
num_actions = 2 ** num_action

save_model = True
load = False
skip_learning = False
watch = True
#dist_training = False

In [27]:
horsetimestart = time()

In [28]:
# Configuration file path
config_file_path = "/home/spillingvoid/programs/Roger/scenarios/rocket_basic.cfg"
model_savefolder = "/home/spillingvoid/programs/Roger/model"
model_weight = "/home/spillingvoid/programs/Roger/weights"
model_callback = "/home/spillingvoid/programs/Roger/callback"
model_statistics = "/home/spillingvoid/programs/Roger/statistics"
videos = "/home/spillingvoid/programs/Roger/statistics"
plots = "/home/spillingvoid/programs/Roger/statistics"
checkpoint_dir = '/home/spillingvoid/programs/Roger/checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [29]:
def preprocess(img):
    img = cv2.resize(img, resolution)
    #img = cv2.cuda_GPUMatt
    #img.upload(img)
    zog = cv2.resize(img, (400, 300))
    
    cv2.imshow("PreProcess", zog)
    img = cv2.Canny(img, 100, 200, L2gradient=True)
    
    zag = cv2.resize(img, (640, 480))
    cv2.imshow("screenshot", zag)
    cv2.waitKey(10)
    #cv2.destroyWindow(img)
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=-1)
    img /= 255
    #channels = img.shape[2]
    #print("Number of Channels: ", channels)
   
    return tf.stack(img)


In [30]:
def initialize_game():
    print("Initializing doom...")
    game = vzd.DoomGame()
    game.load_config(config_file_path)
    game.set_window_visible(False)
    game.set_mode(vzd.Mode.PLAYER)
    game.set_screen_format(vzd.ScreenFormat.GRAY8)
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.get_available_buttons_size()
    game.init()
    print("Doom initialized.")

    return game



In [31]:
def split_tuple(samples):
    samples = np.array(samples, dtype=object)
    screen_buf = tf.stack(samples[:,0])
    actions = samples[:,1]
    rewards = tf.stack(samples[:,2])
    next_screen_buf = tf.stack(samples[:,3])
    dones = tf.stack(samples[:,4])  
    return screen_buf, actions, rewards, next_screen_buf, dones 

In [32]:
replay_memory = deque(maxlen=replay_memory_size)

In [33]:

def extractDigits(*argv):
    
    if len(argv)==1:
        return list(map(lambda x: [x], argv[0]))

    return list(map(lambda x,y: [x,y], argv[0], argv[1]))

In [34]:

def get_samples(memory):
    
    if len(memory) < batch_size:
        sample_size = len(memory)
    else:
        sample_size = batch_size

    return sample(memory, sample_size)

In [35]:

class DQNAgent:
    def __init__(self, num_actions=num_actions, epsilon=1, epsilon_min=0.1, epsilon_decay=0.9999, load=load):
        
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.discount_factor = discount_factor
        self.num_actions = num_actions
        self.optimizer = SGD(learning_rate * hvd.size())
        tf.keras.models.load_model(model_savefolder)
        checkpoint_dir = '/home/spillingvoid/programs/Roger/checkpoints'
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
        checkpoint = tf.train.Checkpoint(model=DQN(self.num_actions), 
                                         optimizer=self.optimizer)

        print("Loading model from: ", model_savefolder) 
        self.dqn = tf.keras.models.load_model(model_savefolder)
        
        if load:
            print("Loading model from: ", model_savefolder) 
            self.dqn = tf.keras.models.load_model(model_savefolder)
        else:
            self.dqn = DQN(self.num_actions)
            self.target_net = DQN(self.num_actions)

    def update_target_net(self):
        self.target_net.set_weights(self.dqn.get_weights())
        
    def choose_action(self, state):
        
        if self.epsilon < np.random.uniform(0,1):
            action = int(tf.argmax(self.dqn(tf.reshape(state, (1,60,90,2))), axis=1))
        else:
            action = np.random.choice(range(self.num_actions), 1)[0]

        return action
        
    def train_dqn(self, samples):
        screen_buf, actions, rewards, next_screen_buf, dones = split_tuple(samples)

        row_ids = list(range(screen_buf.shape[0]))
        ids = extractDigits(row_ids, actions)
        done_ids = extractDigits(np.where(dones)[0])
        
            
        with tf.GradientTape(persistent=False) as tape:  # persistent=True #
            
            
            
            #tape.watch(self.dqn.trainable_variables)

            Q_prev = tf.gather_nd(self.dqn(screen_buf), ids)
            Q_next = self.target_net(next_screen_buf)
            Q_next = tf.gather_nd(Q_next, extractDigits(row_ids, #best_next_action
                            tf.argmax(agent.dqn(next_screen_buf), axis=1)))
            q_target = rewards + self.discount_factor * Q_next #next best q values
            
            if len(done_ids)>0:
                done_rewards = tf.gather_nd(rewards, done_ids)
                q_target = tf.tensor_scatter_nd_update(tensor=q_target, 
                                indices=done_ids, updates=done_rewards) #target q values #2
                        
            td_error = tf.keras.losses.MSE(q_target, Q_prev)
        tape = hvd.DistributedGradientTape(tape)                                        
        gradients = tape.gradient(td_error, self.dqn.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.dqn.trainable_variables),
                                       experimental_aggregate_gradients=True)
            
        for V in range(learning_steps_per_epoch):
            if V == 0:
                hvd.broadcast_variables(self.dqn.trainable_variables, root_rank=0)
                hvd.broadcast_variables(self.optimizer.variables(), root_rank=0)
        if batch_size % 10 == 0 and hvd.local_rank() == 0:
            print('Step #%d\tLoss: %.6f' % (batch, td_error))
        return td_error
        if self.epsilon > self.epsilon_min:
                    self.epsilon *= self.epsilon_decay
        else:
                    self.epsilon = self.epsilon_min
    #for batch_size, (learning_steps_per_epoch // hvd.size()):
    #td_error = (self.train_dqn(samples), batch_size == 0)
            
        #if batch_size % 10 == 0 and hvd.local_rank() == 0:
            #print('Step #%d\tLoss: %.6f' % (batch, td_error))
        if hvd.rank() == 0:
            checkpoint.save(checkpoint_dir)
        
        
    #with mirrored_strategy.scope():
        #@tf.function
        def distributed_train_step(self, samples):
            per_replica_losses = mirrored_strategy.run(self.train_dqn, args=(samples,)) #mirrored_strategy.run
            if epoch % 2 == 0:

                return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM,
                                per_replica_losses, axis=None)
        

In [36]:
def run(agent, game, replay_memory):
    time_start = time()

    for episode in range(num_train_epochs):
        train_scores = []
        print("\nEpoch %d\n-------" % (episode + 1))

        game.new_episode()

        for i in trange(learning_steps_per_epoch, leave=False):
            state = game.get_state()
            screen_buf = preprocess(state.screen_buffer)
            action = agent.choose_action(screen_buf)
            reward = game.make_action(actions[action], frames_per_action)
            done = game.is_episode_finished()

            if not done:
                next_screen_buf = preprocess(game.get_state().screen_buffer)
            else:
                next_screen_buf = tf.zeros(shape=screen_buf.shape)

            if done:
                train_scores.append(game.get_total_reward())

                game.new_episode()

            replay_memory.append((screen_buf, action, reward, next_screen_buf, done))

            if i >= batch_size:
                agent.train_dqn(get_samples(replay_memory)) #agent.distributed_train_step(get_samples(replay_memory)) #
       
            if ((i % target_net_update_steps) == 0):
                agent.update_target_net()

        train_scores = np.array(train_scores)
        #print("Results: mean: %.1f±%.1f," % (train_scores.mean(), train_scores.std()), \
        #          "min: %.1f," % train_scores.min(), "max: %.1f," % train_scores.max())

        test(test_episodes_per_epoch, game, agent)
        print("Total elapsed time: %.2f minutes" % ((time() - time_start) / 60.0))

In [37]:
def test(test_episodes_per_epoch, game, agent):
    test_scores = []
    #with mirrored_strategy.scope():
    print("\nTesting...")
    for test_episode in trange(test_episodes_per_epoch, leave=False):
        game.new_episode()
        while not game.is_episode_finished():
            state = preprocess(game.get_state().screen_buffer)
            best_action_index = agent.choose_action(state)
            game.make_action(actions[best_action_index], frames_per_action)

        r = game.get_total_reward()
        test_scores.append(r)

    test_scores = np.array(test_scores)
        #print("Results: mean: %.1f±%.1f," % (
        #    test_scores.mean(), test_scores.std()), "min: %.1f" % test_scores.min(),
        #      "max: %.1f" % test_scores.max())
    print("reward", r)

In [38]:
class DQN(tf.keras.Model):
    
    def __init__(self, num_actions):
        super(DQN, self).__init__()
        #self.input = tf.keras.Input(shape=(1, 60, 90, 2), dtype = tf.float32)
        self.conv1 = tf.keras.layers.Conv2D(512, kernel_size=10, strides=4, 
                                            input_shape=(1, 60, 90, 2), 
                                            activation ="elu", padding="same")
        self.norm1 = tf.keras.layers.BatchNormalization()
        
        self.conv3 = tf.keras.layers.Conv2D(256, kernel_size=8, strides=3, 
                                            activation ="elu", padding="same")
        self.conv5 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=2,
                                            activation ="elu", padding="same") #input_shape=(9, 14, 8)
        self.drop2 = tf.keras.layers.Dropout(0.5)
        self.conv2 = tf.keras.layers.Conv2D(1024, kernel_size=2, strides=1,
                                            activation ="elu", padding="same")
        self.elu   = tf.keras.layers.ELU()

        self.flatten = tf.keras.layers.Flatten()
        self.hidden1 = tf.keras.layers.Dense(128, activation="elu")
        self.state_value = tf.keras.layers.Dense(1, activation="softmax")
        self.advantage = tf.keras.layers.Dense(num_actions)
    

    @tf.function( experimental_relax_shapes=True)
    def call(self, x, training=True):
        
        
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.conv3(x)
        x = self.conv5(x)
        x = self.drop2(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.hidden1(x)
        
        x1 = x[:, :96]
        x2 = x[:, 96:]
        x1 = self.state_value(x1)
        x2 = self.advantage(x2)
        
        x = x1 + (x2 - tf.reshape(tf.math.reduce_mean(x2, axis=1), shape=(-1,1))) #Q_values
        
        return x
        model=DQN(num_actions)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                         loss=tf.keras.losses.MeanSquaredError())
        tf.keras.callbacks.TensorBoard(log_dir=model_statistics, histogram_freq=1,
                            write_graph=1, write_images=True, update_freq=1, 
                            embeddings_freq=EPOCHS)
        model.summary()
        #model = DQN(num_actions)
        #keras.utils.plot_model(model, 'Doomddddqn.png', show_shapes=True)
        #model.save(model_savefolder)
        #model.save_weights(model_weight)



In [ ]:
if __name__ == '__main__':
    
    maxlen = replay_memory_size
    agent = DQNAgent()
    game = initialize_game()
    replay_memory = deque(maxlen=replay_memory_size)
    n = game.get_available_buttons_size()
    actions = [list(a) for a in it.product([0, 1], repeat=n)]
    
    #with tf.device(DEVICE):
    
    if not skip_learning:
        print("Starting the training!")

        run(agent, game, replay_memory)

        game.close()
        print("======================================")
        print("Training is finished.")

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

  0%|          | 1/2048 [00:00<04:51,  7.02it/s]

Doom initialized.
Starting the training!

Epoch 1
-------


  1%|          | 13/2048 [00:00<02:21, 14.35it/s]

Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/repl

...

  0%|          | 0/4 [00:00<?, ?it/s]              


Testing...


  0%|          | 1/2048 [00:00<03:35,  9.52it/s]

reward 49.0
Total elapsed time: 1093.34 minutes

Epoch 251
-------


  3%|▎         | 67/2048 [00:06<04:25,  7.46it/s]

In [ ]:
            if save_model:
                agent.dqn.save(model_savefolder)
                agent.dqn.save_weights(model_savefolder)
                horsetimestop = time()
                print("Horse Time %.2f minutes" % ((horsetimestop - horsetimestart) / 60))
            game.close()

In [ ]:
        if watch:
            game.set_window_visible(True)
            game.set_mode(vzd.Mode.ASYNC_PLAYER)
            game.init()

            for _ in range(episodes_to_watch):
                game.new_episode()
                while not game.is_episode_finished():
                    state = preprocess(game.get_state().screen_buffer)
                    best_action_index = agent.choose_action(state)

                    # Instead of make_action(a, frame_repeat) in order to make the animation smooth
                    game.set_action(actions[best_action_index])
                    for _ in range(frames_per_action):
                        game.advance_action()

                # Sleep between episodes
                sleep(1.0)
                score = game.get_total_reward()
                print("Total score: ", score)